In [1]:
import xml.etree.ElementTree as ET

def rescale_cvat_annotations(xml_file_path, scale_factor=2.0):
    """
    Rescales annotations in a CVAT XML file.

    Args:
        xml_file_path (str): Path to the input CVAT XML file.
        scale_factor (float): Factor by which to scale the annotations (e.g., 2.0 for 2x upscale).

    Returns:
        str: The modified XML content as a string.
              Returns None if the file cannot be parsed.
    """
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        return None

    # Iterate through each image in the XML
    for image_tag in root.findall('.//image'):
        # Rescale image width and height
        try:
            original_width = int(image_tag.get('width'))
            original_height = int(image_tag.get('height'))
            image_tag.set('width', str(int(original_width * scale_factor)))
            image_tag.set('height', str(int(original_height * scale_factor)))
        except (ValueError, TypeError) as e:
            print(f"Warning: Could not rescale dimensions for image {image_tag.get('name')}: {e}")
            continue # Skip to the next image if dimensions are invalid

        # Rescale polygons
        for polygon_tag in image_tag.findall('.//polygon'):
            try:
                points_str = polygon_tag.get('points')
                if not points_str:
                    continue

                scaled_points = []
                point_pairs = points_str.split(';')
                for pair_str in point_pairs:
                    if not pair_str.strip(): # Skip empty pairs if any
                        continue
                    coords = pair_str.split(',')
                    if len(coords) == 2:
                        x = float(coords[0])
                        y = float(coords[1])
                        scaled_x = round(x * scale_factor, 2) # Round to 2 decimal places
                        scaled_y = round(y * scale_factor, 2)
                        scaled_points.append(f"{scaled_x},{scaled_y}")
                    else:
                        print(f"Warning: Skipping malformed coordinate pair '{pair_str}' in image {image_tag.get('name')}")
                polygon_tag.set('points', ';'.join(scaled_points))
            except (ValueError, TypeError) as e:
                print(f"Warning: Could not rescale points for a polygon in image {image_tag.get('name')}: {e}")


        # Rescale bounding boxes
        for box_tag in image_tag.findall('.//box'):
            try:
                xtl = float(box_tag.get('xtl'))
                ytl = float(box_tag.get('ytl'))
                xbr = float(box_tag.get('xbr'))
                ybr = float(box_tag.get('ybr'))

                box_tag.set('xtl', str(round(xtl * scale_factor, 2)))
                box_tag.set('ytl', str(round(ytl * scale_factor, 2)))
                box_tag.set('xbr', str(round(xbr * scale_factor, 2)))
                box_tag.set('ybr', str(round(ybr * scale_factor, 2)))
            except (ValueError, TypeError) as e:
                 print(f"Warning: Could not rescale attributes for a box in image {image_tag.get('name')}: {e}")


    # Convert the modified tree back to a string
    # ET.indent(tree, space="\t", level=0) # For pretty printing, requires Python 3.9+
    # For compatibility with older Python versions, you can use a more manual pretty print or just output as is.
    # The following line just converts to string without extra pretty printing.
    modified_xml_string = ET.tostring(root, encoding='unicode')
    
    # A simple way to somewhat pretty print for older versions (basic indentation)
    # This is a very basic pretty print, for more complex needs, consider lxml
    from xml.dom import minidom
    rough_string = ET.tostring(root, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ", encoding="utf-8").decode('utf-8')


# --- How to use the script ---

# 1. Save your CVAT XML content to a file named 'gowanmarsh.xml' in the same directory as the script.
#    Or, provide the correct path to your XML file.
input_xml_file = r"C:\Users\kevin\Documents\gowanmarsh.xml" # Or the full path to your file
output_xml_file = 'gowanmarsh_rescaled.xml'
scale_factor = 2.0

# Make sure the input file exists
try:
    with open(input_xml_file, 'r') as f:
        # You can read the content to verify if needed, but the function will parse it.
        pass
    print(f"Successfully located input file: {input_xml_file}")
except FileNotFoundError:
    print(f"Error: The input file '{input_xml_file}' was not found.")
    print("Please ensure the XML file is in the correct location or update the 'input_xml_file' variable.")
    exit()


# Rescale the annotations
modified_xml_content = rescale_cvat_annotations(input_xml_file, scale_factor)

if modified_xml_content:
    # Save the modified XML to a new file
    with open(output_xml_file, 'w', encoding='utf-8') as f:
        f.write(modified_xml_content)
    print(f"Rescaled annotations saved to: {output_xml_file}")
    print("You can now import this file back into CVAT or use it with your upscaled images.")
else:
    print("Failed to rescale annotations.")

Successfully located input file: C:\Users\kevin\Documents\gowanmarsh.xml
Rescaled annotations saved to: gowanmarsh_rescaled.xml
You can now import this file back into CVAT or use it with your upscaled images.
